In [1]:
import pandas as pd
import numpy as np
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, LayerNormalization, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from numpy.random import seed

In [2]:
Xl = pd.read_csv('Xl.csv',header=None)
Xs = pd.read_csv('Xs.csv',header=None)
R = pd.read_csv('y.csv',header = None)

In [3]:
Xl

,0,1,2,3,4,5,6,7,8,9,...,127,128,129,130,131,132,133,134,135,136
0,-3.451521,-2.928627,0.000585,0.502896,0.027736,0.0282,-0.0362,0.0075,0.003460,2437.296,...,11.358,2.13,2.45,2.04,86.897162,274.9,6476.00,12298.00,84.2043,19.214685
1,-3.447556,-2.903999,0.000845,0.494946,0.027899,0.0270,-0.0153,0.0075,-0.003448,2446.902,...,11.375,2.14,2.46,2.05,86.897162,276.0,6476.00,12298.00,83.5280,19.214685
2,-3.444322,-2.880679,0.000492,0.516860,0.024984,0.0280,-0.0263,0.0072,0.000000,2462.689,...,11.395,2.15,2.45,2.07,86.897162,277.4,6508.00,12349.00,81.6405,19.214685
3,-3.478608,-2.888116,0.000493,0.498597,0.024361,0.0295,-0.0412,0.0063,0.003460,2478.744,...,11.436,2.16,2.47,2.08,86.897162,278.1,6620.00,12484.00,81.8099,19.214685
4,-3.493617,-2.877168,0.000428,0.483077,0.025887,0.0284,-0.0289,0.0059,0.000000,2493.228,...,11.454,2.17,2.48,2.08,95.300000,280.1,6753.00,12646.00,80.7315,19.214685
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
739,-4.080892,-3.569975,0.000743,0.235975,-0.008504,0.0010,-0.0359,0.0102,0.003153,17603.151,...,120.338,25.51,29.39,22.87,74.100000,21074.4,344023.25,726723.85,4367.8773,20.277700
740,-4.045576,-3.533379,0.004907,0.241482,-0.005698,0.0011,0.0069,0.0105,0.001393,17698.892,...,120.743,25.49,29.09,23.01,80.400000,21249.9,347627.43,730734.42,4425.5453,28.324100
741,-4.020767,-3.519301,0.003661,0.253146,-0.001895,0.0010,-0.0248,0.0109,0.000415,17573.127,...,120.871,25.58,29.40,22.99,81.800000,21369.3,348262.68,730398.69,4505.3741,30.940800
742,-4.126172,-3.635623,0.002492,0.226352,-0.005262,0.0009,0.0084,0.0100,-0.000611,17355.669,...,120.874,25.69,29.55,23.10,76.900000,21565.0,350766.10,733096.73,4609.7215,24.804700


In [4]:
Xs

,0,1,2,3,4,5,6,7,8
0,-3.451521,-2.928627,0.000585,0.502896,0.027736,0.0282,-0.0362,0.0075,0.003460
1,-3.447556,-2.903999,0.000845,0.494946,0.027899,0.0270,-0.0153,0.0075,-0.003448
2,-3.444322,-2.880679,0.000492,0.516860,0.024984,0.0280,-0.0263,0.0072,0.000000
3,-3.478608,-2.888116,0.000493,0.498597,0.024361,0.0295,-0.0412,0.0063,0.003460
4,-3.493617,-2.877168,0.000428,0.483077,0.025887,0.0284,-0.0289,0.0059,0.000000
...,...,...,...,...,...,...,...,...,...
739,-4.080892,-3.569975,0.000743,0.235975,-0.008504,0.0010,-0.0359,0.0102,0.003153
740,-4.045576,-3.533379,0.004907,0.241482,-0.005698,0.0011,0.0069,0.0105,0.001393
741,-4.020767,-3.519301,0.003661,0.253146,-0.001895,0.0010,-0.0248,0.0109,0.000415
742,-4.126172,-3.635623,0.002492,0.226352,-0.005262,0.0009,0.0084,0.0100,-0.000611


In [5]:
# R

In [6]:
y_agg=R.iloc[::-1].rolling(window=12).sum().iloc[::-1]

In [7]:
t_train_start = list(range(46*12))
t_train_end =[x+120 for x in t_train_start]
t_val_start= [x for x in t_train_end]
t_val_end = [x+60 for x in t_val_start]
t_test_start = [x for x in t_val_end]
t_test_end = [x for x in t_test_start]

In [8]:
np.shape(t_test_end)

(552,)

In [9]:
t_test_end[551]

731

In [10]:
# optimizer = keras.optimizers.Adam(0.0001)
# what if verbose=1? DONE!
# try learning rate and patience
optimizer=keras.optimizers.RMSprop(0.0005)

loss = []
preds = []
seed(1)
for retrain_idx in range(552):
    X = Xs.iloc[t_train_start[retrain_idx]:t_train_end[retrain_idx],:]
    X_idx = X.apply(pd.Series.nunique) != 1
    X = X.loc[:,X_idx]
    X_val = Xs.loc[t_val_start[retrain_idx]:t_val_end[retrain_idx],X_idx]
    X_test = Xs.loc[t_test_start[retrain_idx]:t_test_end[retrain_idx],X_idx]
    y = y_agg.iloc[t_train_start[retrain_idx]:t_train_end[retrain_idx],:]
    y_val = y_agg.loc[t_val_start[retrain_idx]:t_val_end[retrain_idx],:]
    y_test = y_agg.loc[t_test_start[retrain_idx]:t_test_end[retrain_idx],:]
  
    model = Sequential()
    model.add(LayerNormalization())
    model.add(Dense(512,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(512,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(512,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(512,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(512,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(512,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(512,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(512,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(1))
    model.compile(optimizer=optimizer,loss='mse')
    early_stop = EarlyStopping(monitor='val_loss', verbose=1, patience=15)
    history = model.fit(x=X,y=y,validation_data=(X_val,y_val), batch_size=512, epochs=40,verbose=1,callbacks=[early_stop])
    
    preds.append(model.predict(X_test))
    loss.append(min(history.history['val_loss']))

Epoch 1/40

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

1/1 [==============================] - 0s 426ms/step - loss: 1.8747 - val_loss: 0.0327
Epoch 2/40
1/1 [==============================] - 0s 25ms/step - loss: 16.7953 - val_loss: 0.0334
Epoch 3/40
1/1 [==============================] - 0s 25ms/step - loss: 14.1593 - val_loss: 0.0364
Epoch 4/40
1/1 [==============================] - 0s 28ms/step - loss: 12.6898 - val_loss: 0.0373
Epoch 5/40
1/1 [==============================] - 0s 32ms/step - loss: 8.0571 - val_loss: 0.0402
Epoch 6/40
1/1 [===

1/1 [==============================] - 0s 33ms/step - loss: 7.4911 - val_loss: 0.0393
Epoch 7/40
1/1 [==============================] - 0s 24ms/step - loss: 5.3887 - val_loss: 0.0388
Epoch 8/40
1/1 [==============================] - 0s 25ms/step - loss: 3.9791 - val_loss: 0.0409
Epoch 9/40
1/1 [==============================] - 0s 24ms/step - loss: 2.9911 - val_loss: 0.0404
Epoch 10/40
1/1 [==============================] - 0s 40ms/step - loss: 2.6103 - val_loss: 0.0400
Epoch 11/40
1/1 [==============================] - 0s 40ms/step - loss: 1.4238 - val_loss: 0.0414
Epoch 12/40
1/1 [==============================] - ETA: 0s - loss: 0.770 - 0s 39ms/step - loss: 0.7705 - val_loss: 0.0395
Epoch 13/40
1/1 [==============================] - 0s 49ms/step - loss: 0.3495 - val_loss: 0.0414
Epoch 14/40
1/1 [==============================] - 0s 49ms/step - loss: 0.1391 - val_loss: 0.0401
Epoch 15/40
1/1 [==============================] - 0s 43ms/step - loss: 0.0770 - val_loss: 0.0414
Epoch 16/40

1/1 [==============================] - 0s 40ms/step - loss: 14.7657 - val_loss: 0.0323
Epoch 3/40
1/1 [==============================] - 0s 32ms/step - loss: 16.4143 - val_loss: 0.0333
Epoch 4/40
1/1 [==============================] - 0s 71ms/step - loss: 13.5795 - val_loss: 0.0325
Epoch 5/40
1/1 [==============================] - 0s 49ms/step - loss: 8.4827 - val_loss: 0.0337
Epoch 6/40
1/1 [==============================] - 0s 34ms/step - loss: 7.3945 - val_loss: 0.0336
Epoch 7/40
1/1 [==============================] - 0s 36ms/step - loss: 5.9615 - val_loss: 0.0340
Epoch 8/40
1/1 [==============================] - 0s 31ms/step - loss: 4.0399 - val_loss: 0.0339
Epoch 9/40
1/1 [==============================] - 0s 31ms/step - loss: 2.9297 - val_loss: 0.0346
Epoch 10/40
1/1 [==============================] - 0s 25ms/step - loss: 1.6746 - val_loss: 0.0345
Epoch 11/40
1/1 [==============================] - 0s 24ms/step - loss: 1.1084 - val_loss: 0.0351
Epoch 12/40
1/1 [===================

Epoch 8/40
1/1 [==============================] - ETA: 0s - loss: 3.170 - 0s 40ms/step - loss: 3.1702 - val_loss: 0.0303
Epoch 9/40
1/1 [==============================] - 0s 45ms/step - loss: 2.3757 - val_loss: 0.0304
Epoch 10/40
1/1 [==============================] - 0s 44ms/step - loss: 0.9066 - val_loss: 0.0303
Epoch 11/40
1/1 [==============================] - 0s 39ms/step - loss: 0.4330 - val_loss: 0.0302
Epoch 12/40
1/1 [==============================] - 0s 39ms/step - loss: 0.3463 - val_loss: 0.0304
Epoch 13/40
1/1 [==============================] - 0s 39ms/step - loss: 0.3909 - val_loss: 0.0303
Epoch 14/40
1/1 [==============================] - 0s 36ms/step - loss: 0.4987 - val_loss: 0.0309
Epoch 15/40
1/1 [==============================] - 0s 49ms/step - loss: 0.6044 - val_loss: 0.0303
Epoch 16/40
1/1 [==============================] - 0s 41ms/step - loss: 0.6536 - val_loss: 0.0318
Epoch 17/40
1/1 [==============================] - 0s 49ms/step - loss: 0.5799 - val_loss: 0.030

Epoch 1/40

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

1/1 [==============================] - 0s 265ms/step - loss: 1.9809 - val_loss: 0.0331
Epoch 2/40
1/1 [==============================] - 0s 21ms/step - loss: 17.7961 - val_loss: 0.0328
Epoch 3/40
1/1 [==============================] - 0s 39ms/step - loss: 12.6778 - val_loss: 0.0331
Epoch 4/40
1/1 [==============================] - 0s 40ms/step - loss: 11.2481 - val_loss: 0.0331
Epoch 5/40
1/1 [==============================] - 0s 45ms/step - loss: 9.6531 - val_loss: 0.0332
Epoch 6/40
1/1 [===

...

Epoch 2/40
1/1 [==============================] - 0s 30ms/step - loss: 10.9805 - val_loss: 0.0095
Epoch 3/40
1/1 [==============================] - 0s 16ms/step - loss: 19.1723 - val_loss: 0.0061
Epoch 4/40
1/1 [==============================] - 0s 34ms/step - loss: 8.5442 - val_loss: 0.0062
Epoch 5/40
1/1 [==============================] - 0s 24ms/step - loss: 10.2562 - val_loss: 0.0061
Epoch 6/40
1/1 [==============================] - 0s 31ms/step - loss: 6.1978 - val_loss: 0.0064
Epoch 7/40
1/1 [==============================] - 0s 25ms/step - loss: 4.7186 - val_loss: 0.0061
Epoch 8/40
1/1 [==============================] - 0s 22ms/step - loss: 3.2817 - val_loss: 0.0066
Epoch 9/40
1/1 [==============================] - 0s 23ms/step - loss: 3.7536 - val_loss: 0.0058
Epoch 10/40
1/1 [==============================] - 0s 16ms/step - loss: 2.7542 - val_loss: 0.0060
Epoch 11/40
1/1 [==============================] - 0s 33ms/step - loss: 2.1389 - val_loss: 0.0057
Epoch 12/40
1/1 [========

Epoch 1/40

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

1/1 [==============================] - 0s 290ms/step - loss: 2.5012 - val_loss: 0.0071
Epoch 2/40
1/1 [==============================] - 0s 34ms/step - loss: 9.7153 - val_loss: 0.0066
Epoch 3/40
1/1 [==============================] - 0s 26ms/step - loss: 19.0952 - val_loss: 0.0060
Epoch 4/40
1/1 [==============================] - 0s 22ms/step - loss: 12.8797 - val_loss: 0.0061
Epoch 5/40
1/1 [==============================] - 0s 22ms/step - loss: 11.1820 - val_loss: 0.0058
Epoch 6/40
1/1 [===

Epoch 1/40

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

1/1 [==============================] - 0s 310ms/step - loss: 1.8374 - val_loss: 0.0113
Epoch 2/40
1/1 [==============================] - 0s 27ms/step - loss: 7.8266 - val_loss: 0.0061
Epoch 3/40
1/1 [==============================] - 0s 34ms/step - loss: 14.7918 - val_loss: 0.0061
Epoch 4/40
1/1 [==============================] - 0s 37ms/step - loss: 10.2256 - val_loss: 0.0067
Epoch 5/40
1/1 [==============================] - 0s 23ms/step - loss: 7.9270 - val_loss: 0.0057
Epoch 6/40
1/1 [====

1/1 [==============================] - 0s 25ms/step - loss: 2.8453 - val_loss: 0.0072
Epoch 11/40
1/1 [==============================] - 0s 26ms/step - loss: 2.4226 - val_loss: 0.0067
Epoch 12/40
1/1 [==============================] - 0s 21ms/step - loss: 1.6369 - val_loss: 0.0069
Epoch 13/40
1/1 [==============================] - 0s 23ms/step - loss: 1.2368 - val_loss: 0.0073
Epoch 14/40
1/1 [==============================] - 0s 19ms/step - loss: 1.1728 - val_loss: 0.0070
Epoch 15/40
1/1 [==============================] - 0s 21ms/step - loss: 1.1035 - val_loss: 0.0080
Epoch 16/40
1/1 [==============================] - 0s 20ms/step - loss: 0.8323 - val_loss: 0.0076
Epoch 17/40
1/1 [==============================] - 0s 28ms/step - loss: 0.4810 - val_loss: 0.0085
Epoch 18/40
1/1 [==============================] - 0s 32ms/step - loss: 0.3280 - val_loss: 0.0080
Epoch 19/40
1/1 [==============================] - 0s 64ms/step - loss: 0.2794 - val_loss: 0.0090
Epoch 20/40
1/1 [===============

Epoch 19/40
1/1 [==============================] - 0s 21ms/step - loss: 0.5226 - val_loss: 0.0085
Epoch 20/40
1/1 [==============================] - 0s 29ms/step - loss: 0.6087 - val_loss: 0.0083
Epoch 00020: early stopping


In [11]:
y_org = y_agg.loc[t_test_start[0]:t_test_end[551],:]

### Calculate R^2

In [12]:
1-np.sum((np.squeeze(y_org)-np.squeeze(preds))**2)/np.sum((y_org-np.mean(y_org))**2)

0   -0.307925
dtype: float64

In [13]:
y_org

,0
180,-0.277289
181,-0.145559
182,-0.080091
183,-0.034259
184,0.052814
...,...
727,0.140529
728,0.228682
729,0.171740
730,0.123710


### A good R^2 is expected to be between 0 to 0.05.